# Flux sampling

## Basic usage

The easiest way to get started with flux sampling is using the `sample` function in the `flux_analysis` submodule. `sample` takes at least two arguments: a cobra model and the number of samples you want to generate.

In [1]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample

model = create_test_model("textbook")
s = sample(model, 100)
s.head()

,ACALD,ACALDt,ACKr,ACONTa,ACONTb,ACt2r,ADK1,AKGDH,AKGt2r,ALCD2x,...,RPI,SUCCt2_2,SUCCt3,SUCDi,SUCOAS,TALA,THD2,TKT1,TKT2,TPI
0,-0.138418,-0.007443,-0.140351,5.815865,5.815865,-0.140351,7.061811,0.661809,-0.600300,-0.130976,...,-4.835818,7.330825,8.560406,638.055628,-0.661809,4.835438,7.344945,4.835438,4.835286,4.375830
1,-1.186611,-0.151028,-0.743592,6.779593,6.779593,-0.743592,7.071247,2.681170,-0.612076,-1.035583,...,-3.816782,16.226150,17.127049,594.924213,-2.681170,3.812960,20.987681,3.812960,3.811424,5.395978
2,-1.126487,-0.158074,-0.832578,7.449216,7.449216,-0.832578,10.441861,3.959961,-0.648730,-0.968413,...,-5.577614,4.154524,4.270371,551.412357,-3.959961,5.573537,27.635521,5.573537,5.571897,3.739413
3,-1.252806,-0.148776,-0.243209,7.129622,7.129622,-0.243209,10.979313,0.492235,-1.489826,-1.104031,...,-5.599746,13.425704,13.801661,841.109531,-0.492235,5.582190,10.129844,5.582190,5.575129,4.093422
4,-0.511033,-0.246761,-0.042680,8.146012,8.146012,-0.042680,9.205141,3.021588,-1.857814,-0.264272,...,-3.919012,10.950205,11.164086,796.765674,-3.021588,3.905839,1.932595,3.905839,3.900542,4.584599


By default sample uses the `optgp` method based on the [method presented here](http://dx.doi.org/10.1371/journal.pone.0086587) as it is suited for larger models and can run in parallel. By default the sampler uses a single process. This can be changed by using the `processes` argument.

In [2]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)

One process:
CPU times: user 5.07 s, sys: 497 ms, total: 5.57 s
Wall time: 5.03 s
Two processes:
CPU times: user 246 ms, sys: 642 ms, total: 888 ms
Wall time: 2.63 s


Alternatively you can also user Artificial Centering Hit-and-Run for sampling by setting the method to `arch`. `arch` does not support parallel execution but has good convergence and is almost Markovian.

In [3]:
s = sample(model, 100, method="arch")

In general setting up the sampler is expensive since initial search directions are generated by solving many linear programming problems. Thus, we recommend to generate as many samples as possible in one go. However, this might require finer control over the sampling procedure as described in the following section.

## Advanced usage

### Sampler objects

The sampling process can be controlled on a lower level by using the sampler classes directly.

In [4]:
from cobra.flux_analysis.sampling import OptGPSampler, ARCHSampler

Both sampler classes have standardized interfaces and take some additional argument. For instance the `thinning` factor. "Thinning" means only recording samples every n iterations. A higher thinning factors mean less correlated samples but also larger computation times. By default the samplers use a thinning factor of 100 which creates roughly uncorrelated samples. If you want less samples but better mixing feel free to increase this parameter. If you want to study convergence for your own model you might want to set it to 1 to obtain all iterates.

In [5]:
arch = ARCHSampler(model, thinning=10)

`OptGPSampler` has an additional `processes` argument specifying how many processes are used to create parallel sampling chains. This should be in the order of your CPU cores for maximum efficiency. As noted before class initialization can take up to a few minutes due to generation of initial search directions. Sampling on the other hand is quick.

In [6]:
optgp = OptGPSampler(model, processes=4)

### Sampling and validation

Both samplers have a sample function that generates samples from the initialized object and act like the `sample` function described above, only that this time it will only accept a single argument, the number of samples. For `OptGPSampler` the number of samples should be a multiple of the number of processes, otherwise it will be increased to the nearest multiple automatically.

In [7]:
s1 = arch.sample(100)

s2 = optgp.sample(100)

You can call `sample` repeatedly and both samplers are optimized to generate large amount of samples without falling into "numerical traps". All sampler objects have a `validate` function in order to check if a set of points are feasible and give detailed information about feasibility violations in a form of a short code denoting feasibility. Here the short code is a combination of any of the following letters:

- "v" - valid point
- "l" - lower bound violation
- "u" - upper bound violation
- "e" - equality violation (meaning the point is not a steady state)

For instance for a random flux distribution (should not be feasible):

In [8]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
arch.validate(np.atleast_2d(bad))

array(['le'], 
      dtype='<U3')

And for our generated samples:

In [9]:
arch.validate(s1)

array(['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v'], 
      dtype='<U3')

### Batch sampling

Sampler objects are made for generating billions of samples, however using the `sample` function might quickly fill up your RAM when working with genome-scale models. Here, the `batch` method of the sampler objects might come in handy. `batch` takes two arguments, the number of samples in each batch and the number of batches. This will make sense with a small example. 

Let's assume we want to quantify what proportion of our samples will grow. For that we might want to generate 10 batches of 50 samples each and measure what percentage of the individual 100 samples show a growth rate larger than 0.1. Finally, we want to calculate the mean and standard deviation of those individual percentages.

In [10]:
counts = [np.mean(s.Biomass_Ecoli_core > 0.1) for s in optgp.batch(100, 10)]
print("Usually {:.2f}% +- {:.2f}% grow...".format(
    np.mean(counts) * 100.0, np.std(counts) * 100.0))

Usually 5.20% +- 1.72% grow...


## Adding constraints

Flux sampling will respect additional contraints defined in the model. For instance we can add a constraint enforcing growth in asimilar manner as the section before.

In [11]:
co = model.problem.Constraint(model.reactions.Biomass_Ecoli_core.flux_expression, lb=0.1)
model.add_cons_vars([co])

*Note that this is only for demonstration purposes. usually you could set the lower bound of the reaction directly instead of creating a new constraint.*

In [12]:
s = sample(model, 10)
print(s.Biomass_Ecoli_core)

0    0.137251
1    0.148080
2    0.153586
3    0.145562
4    0.150578
5    0.125127
6    0.130563
7    0.139333
8    0.120124
9    0.120522
Name: Biomass_Ecoli_core, dtype: float64


As we can see our new constraint was respected. It should be noted that adding **equality** constraints will strongly affect the performance of the sampler (about 10x slower) due to an additional required projection step in each iteration.